<a href="https://colab.research.google.com/github/Nguyencongdat1997/RL.TryOut/blob/developments-ppo/Simple_TD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

In [1]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]
# import gym
# env = gym.make("LunarLander-v2")

     |████████████████████████████████| 450kB 4.2MB/s 


In [12]:
import gym
import random

In [3]:
env = gym.make('BipedalWalker-v3')
observation_space = env.observation_space.shape
action_space = env.action_space

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [9]:
print(action_space)
sample_action = env.action_space.sample()
print(sample_action)
print(observation_space)
state = env.reset()
print(state)
state, reward, done, info = env.step(sample_action)
print(state, reward, done, info)

Box(-1.0, 1.0, (4,), float32)
[-0.20892707  0.11706837  0.6628375   0.53771186]
(24,)
[ 2.74599786e-03  9.57680924e-06 -1.24876454e-03 -1.60000646e-02
  9.24573764e-02  2.89892987e-03  8.59824702e-01 -9.02140280e-04
  1.00000000e+00  3.27833407e-02  2.89881648e-03  8.53548557e-01
 -1.97249915e-03  1.00000000e+00  4.40813392e-01  4.45819497e-01
  4.61422116e-01  4.89549488e-01  5.34102023e-01  6.02460206e-01
  7.09147871e-01  8.85930538e-01  1.00000000e+00  1.00000000e+00]
[ 3.04660702e-04 -1.17167556e-02  4.73519862e-03  1.74500620e-02
 -2.70614445e-01 -5.17780602e-01  1.45148164e+00  7.85511176e-01
  1.00000000e+00  2.87180662e-01 -3.16510558e-01  1.91316724e-01
  5.21972060e-01  1.00000000e+00  4.55141306e-01  4.60310131e-01
  4.76419896e-01  5.05461514e-01  5.51462173e-01  6.22042179e-01
  7.32197583e-01  9.14726317e-01  1.00000000e+00  1.00000000e+00] -0.02945383042485261 False {}


In [13]:
import numpy as np
import tensorflow as tf

episodes = 3
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    steps = 0
    while not done:
        # env.render()
        mu = np.random.normal(scale=env.action_space.high[0], size=(env.action_space.shape[0]))
        clipped_mu = tf.clip_by_value(mu, env.action_space.low[0], env.action_space.high[0])
        action = clipped_mu
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1
    print('Episode:{} Steps:{} Score:{}'.format(episode, steps, score))
env.close()

Episode:1 Steps:61 Score:-119.17519531884324
Episode:2 Steps:91 Score:-103.2128391900563
Episode:3 Steps:58 Score:-117.55205544809183


# TD3 tf

## Import

In [ ]:
import tensorflow as tf
tf.__version__ #2.5.0

'2.5.0'

In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

## Experience Replay

In [17]:
class ReplayBuffer():
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_counter = 0

        self.states = np.zeros((self.mem_size, *input_shape), dtype=np.float32)
        self.next_states = np.zeros((self.mem_size, *input_shape), dtype=np.float32)
        self.rewards = np.zeros(self.mem_size, dtype=np.float32)
        self.actions = np.zeros((self.mem_size, n_actions), dtype=np.float32)
        self.done = np.zeros(self.mem_size, dtype=np.bool)

    def store_step(self, state, action, reward, next_state, done):
        index = self.mem_counter % self.mem_size
        self.states[index] = state
        self.next_states[index] = next_state
        self.actions[index] = action
        self.rewards[index] = reward
        self.done[index] = done
        self.mem_counter += 1

    def sample_buffer(self, batch_size):
        # if batch_size > self.mem_counter:
        #   return [], [], [], [], []
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.states[batch]
        next_states = self.next_states[batch]
        rewards = self.rewards[batch]
        actions = self.actions[batch]
        dones = self.done[batch]

        return states, actions, rewards, next_states, dones
    

## Network

In [18]:
class Critic(keras.Model):
    def __init__(self):
        super(Critic, self).__init__()
        l1_dims = 400 # As tested in the paper
        l2_dims = 300 # As tested in the paper

        self.fc1 = Dense(l1_dims, activation='relu') #input_shape  = state, action
        self.fc2 = Dense(l2_dims, activation='relu')
        self.q = Dense(1, activation=None)

    def call(self, state, action):
        input = tf.concat([state, action], axis=1)
        x = self.fc1(input)
        x = self.fc2(x)
        q = self.q(x)
        return q

In [19]:
class Actor(keras.Model):
    def __init__(self, n_actions):
        super(Actor, self).__init__()
        l1_dims = 400 # As tested in the paper
        l2_dims = 300 # As tested in the paper
        self.n_actions = n_actions

        self.fc1 = Dense(l1_dims, activation='relu') #input_shape  = state
        self.fc2 = Dense(l2_dims, activation='relu')
        self.mu = Dense(self.n_actions, activation='tanh')

        #self.max_action = max_action

    def call(self, state):        
        x = self.fc1(state)
        x = self.fc2(x)
        mu = self.mu(x)
        # mu *= self.max_action
        return mu

## Agent

In [20]:
class Agent():
    def __init__(self, observation_shape, n_actions, max_action, min_action, 
                 actor_learning_rate=0.001, critic_learning_rate=0.001, tau=0.005, gamma=0.99, 
                 delayed_policy_update_interval=2, exploration_noise_std=0.1, 
                 smoothing_noise_std=0.2, smoothing_noise_limit=0.5,
                 mem_size=1000000, batch_size=100):
        self.n_actions = n_actions
        self.max_action = max_action
        self.min_action = min_action
        self.gamma = gamma        
        self.tau = tau        
        self.delayed_policy_update_interval = delayed_policy_update_interval
        self.exploration_noise_std = exploration_noise_std
        self.smoothing_noise_std = smoothing_noise_std
        self.smoothing_noise_limit = smoothing_noise_limit
        self.batch_size = batch_size               

        self.learned_step_counter = 0        
        self.memory = ReplayBuffer(mem_size, observation_shape, n_actions)
        self.actor = Actor(n_actions)
        self.critic1 = Critic()
        self.critic2 = Critic()
        self.target_actor = Actor(n_actions)
        self.target_critic1 = Critic()
        self.target_critic2 = Critic()

        self.actor.compile(optimizer=Adam(learning_rate=actor_learning_rate), loss='mean')
        self.target_actor.compile(optimizer=Adam(learning_rate=actor_learning_rate), loss='mean')
        self.critic1.compile(optimizer=Adam(learning_rate=critic_learning_rate), loss='mean_squared_error')
        self.critic2.compile(optimizer=Adam(learning_rate=critic_learning_rate), loss='mean_squared_error')
        self.target_critic1.compile(optimizer=Adam(learning_rate=critic_learning_rate), loss='mean_squared_error')
        self.target_critic2.compile(optimizer=Adam(learning_rate=critic_learning_rate), loss='mean_squared_error')
        
        self.soft_update_target_network_parameters(tau=1)    
    
    def choose_action(self, observation, explorating=False):
        # if self.time_step < self.warmup:
        #     mu = np.random.normal(scale=self.exploration_noise_std, size=(self.n_actions)) #size=(self.n_actions,)
        # else:
            state = np.array([observation])
            state = tf.convert_to_tensor(state, dtype=tf.float32)
            mu = self.actor(state)[0]
            if explorating:                
                mu = mu + np.random.normal(scale=self.exploration_noise_std)
                mu = tf.clip_by_value(mu, self.min_action, self.max_action)
            return mu
    
    def store_step(self, state, action, reward, next_state, done):
        self.memory.store_step(state, action, reward, next_state, done)
       
    def run_warmup(self, env, warmup):
        print('Run warmup ----------')       
        steps = 0        
        while steps < warmup:
            done = False
            observation = env.reset()
            while not done:
                steps += 1
                mu = np.random.normal(scale=self.max_action, size=(self.n_actions))
                action = clipped_mu
                next_observation, reward, done, info = env.step(action)
                self.store_step(observation, action, reward, next_observation, done)
                observation = next_observation                  

    def train(self, env, n_games, warmup=1000):
        if warmup > 0:
            self.run_warmup(env, warmup)

        print('Start training ----------')  
        best_score = env.reward_range[0]
        scores = []
        time_steps = 0
        for i in range(n_games):
            done = False
            score = 0
            observation = env.reset()
            while not done:
                time_steps += 1
                action = self.choose_action(observation, explorating=True)
                next_observation, reward, done, info = env.step(action)
                score += reward
                self.store_step(observation, action, reward, next_observation, done)
                self.learn()
                observation = next_observation
                self.learn()

            scores.append(score)
            avg_score = np.mean(scores[-10:])
            if avg_score > best_score:
                best_score = avg_score
                if best_score > 250:
                    self.save_models(train_dir='.')
            print('Episode', i, '- trained steps', time_steps, '- score %.1f'%score, '- avg_score %.1f ' % avg_score)
        print('End training ----------')  

    def learn(self):
        if self.memory.mem_counter < self.batch_size:
            return

        states, actions, rewards, next_states, dones = self.memory.sample_buffer(self.batch_size)
        states = tf.convert_to_tensor(states, dtype=tf.float32)
        actions = tf.convert_to_tensor(actions, dtype=tf.float32)
        rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
        next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)

        with tf.GradientTape(persistent=True) as tape: # <- 2 networks update
            target_actions = self.target_actor(next_states)
            smoothing_noise = tf.clip_by_value(np.random.normal(scale=self.smoothing_noise_std), 
                                    -1*self.smoothing_noise_limit, self.smoothing_noise_limit)            
            target_actions = target_actions + smoothing_noise
            target_actions = tf.clip_by_value(target_actions, self.min_action, self.max_action)

            next_q1 = self.target_critic1(next_states, target_actions)
            next_q1 = tf.squeeze(next_q1, 1) # [batch_size, 1] -> [batch_size]
            next_q2 = self.target_critic2(next_states, target_actions)
            next_q2 = tf.squeeze(next_q2, 1)
            next_q = tf.math.minimum(next_q1, next_q2)

            q1 = self.critic1(states, actions)
            q1 = tf.squeeze(q1, 1) 
            q2 = self.critic2(states, actions)
            q2 = tf.squeeze(q2, 1) 
            
            target = rewards + self.gamma*next_q*(1-dones)

            critic1_loss = keras.losses.MSE(target, q1)
            critic2_loss = keras.losses.MSE(target, q2)
        critic1_gradient = tape.gradient(critic1_loss, self.critic1.trainable_variables)
        critic2_gradient = tape.gradient(critic2_loss, self.critic2.trainable_variables)
        self.critic1.optimizer.apply_gradients(zip(critic1_gradient, self.critic1.trainable_variables))
        self.critic2.optimizer.apply_gradients(zip(critic2_gradient, self.critic2.trainable_variables))

        self.learned_step_counter += 1

        if self.learned_step_counter+1 % self.delayed_policy_update_interval == 0:
            with tf.GradientTape() as tape:
                new_actions = self.actor(states)
                q = self.critic1(states, new_actions)
                actor_loss =  -1 * tf.math.reduce_mean(q)
            actor_gradient = tape.gradient(actor_loss, self.actor.trainable_variables)
            self.actor.optimizer.apply_gradients(zip(actor_gradient, self.actor.trainable_variables))

            self.soft_update_target_network_parameters(tau=self.tau)

    def soft_update_target_network_parameters(self, tau):
        weights = []
        targets = self.target_actor.weights
        for i, weight in enumerate(self.actor.weights):
            weights.append(weight*tau + targets[i]*(1-tau))
        self.target_actor.set_weights(weights)

        weights = []
        targets = self.target_critic1.weights
        for i, weight in enumerate(self.critic1.weights):
            weights.append(weight*tau + targets[i]*(1-tau))
        self.target_critic1.set_weights(weights)

        weights = []
        targets = self.target_critic2.weights
        for i, weight in enumerate(self.critic2.weights):
            weights.append(weight*tau + targets[i]*(1-tau))
        self.target_critic2.set_weights(weights)

    def save_models(self, train_dir):
        pass
        # train_dir = train_dir + '/td3_' + str(self.learned_step_counter) 
        # self.actor.save_weights(train-dir+'/actor/model')
        # self.critic1.save_weights(train-dir+'/critic1/model')
        # self.critic2.save_weights(train-dir+'/critic2/model')
        # self.target_actor.save_weights(train-dir+'/target_actor/model')
        # self.target_critic1.save_weights(train-dir+'/target_critic1/model')
        # self.target_critic2.save_weights(train-dir+'/target_critic2/model')

    def load_models(self, train_dir, learned_steps = 100):
        pass
        # train_dir = train_dir + '/td3_' + str(learned_steps) 
        # self.actor.load_weights(train-dir+'/actor/model')
        # self.critic1.load_weights(train-dir+'/critic1/model')
        # self.critic2.load_weights(train-dir+'/critic2/model')
        # self.target_actor.load_weights(train-dir+'/target_actor/model')
        # self.target_critic1.load_weights(train-dir+'/target_critic1/model')
        # self.target_critic2.load_weights(train-dir+'/target_critic2/model')


## Training

In [ ]:
agent = Agent(observation_shape=env.observation_space.shape, 
              n_actions=env.action_space.shape[0], 
              max_action=env.action_space.high[0], 
              min_action=env.action_space.low[0],
              critic_learning_rate=0.01, actor_learning_rate=0.01, tau=0.1)
agent.train(env, n_games=1000, warmup=1000)

## Testing

In [ ]:
episodes = 3
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    steps = 0
    while not done:
        # env.render()        
        action = agent.choose_action(state)
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1
    print('Episode:{} Steps:{} Score:{}'.format(episode, steps, score))
env.close()

In [ ]:
state = env.reset()
action = agent.choose_action(state)
action

# TD3 pytorch

## Import

## Experience Replay

## Network

## Agent

## Training

## Testing

In [ ]:
#